In [1]:
from __future__ import division
from __future__ import print_function
import os.path
import numpy as np
import tensorflow as tf
import pickle

In [2]:
def read_data():
    content = []
    #test_content = []
    #for i in range(len(interval)):
    #   content.append([])
    #content.append([])

    with open('/media/nikhil/Data/Experiments/ECG Problem/sample rr interval data/line_count_rr_with_category.txt', 'r') as fr:
        rr_name = fr.readlines()[0].split('\r')[:-2]
        for names in rr_name:
            name = names.split(' ')
            #for nam in name:
            #    print nam
            #s = f.rstrip().replace('\r', ' ').split(' ')
            #print s
            #ll=[]
            if name[2] =="Normal":
                with open('/media/nikhil/Data/Experiments/ECG Problem/sample rr interval data/'+name[1], 'r') as rr:
                    l = rr.readlines()
                    #for i, x in enumerate(l):
                    l = [x.replace('\tN\n', '') for x in l]
                    #print l
                    content.append(l)
            #elif name[2] =="Abnormal":
            #    with open('/media/nikhil/Data/Experiments/ECG Problem/sample rr interval data/'+name[1], 'r') as rr:
            #        l = rr.readlines()
                    #for i, x in enumerate(l):
            #        l = [x.replace('\tN\n', '') for x in l]
                    #print l
            #        test_content.append(l)

                    #l = [x.rstrip() for x in l]
            #for i, j in enumerate(interval):
            #    if len(l) >= j:
            #        content[i].append(l)
    #for con in content:
    #    print (len(con))
    return content

In [3]:
def generate_data(arr, length):
    #generate multiple array of fixed length from a list
    #taking window size of 10, can make it variable also
    result = []
    for i in xrange(0, len(arr), 10):
        #print (i)
        if (i+length)>=len(arr):
            break
        result.append(arr[i:i+length])
    #for res in result:
    #    print (len(res))
    return result

In [4]:
def prepare_data(content,length):
    new_content = []
    #np.empty([len(content),20])
    j = 0
    for i, con in enumerate(content):
        if len(con) > (length-1):
            result = generate_data(con, length)
            for res in result:
                new_content.append(res)
                #j = j+1
    return new_content

In [5]:
#preparing data of length of interval 20,

#from tensorflow.examples.tutorials.mnist import input_data

#mnist = input_data.read_data_sets('MNIST')

#set of intervals to try out autoencoder
#input_dim = 784


#index of internal interval(input_dim)
i =0

input_dim = [20, 30, 50, 70]
content = read_data()
new_content = prepare_data(content, input_dim[i])
#abnormal_content = prepare_data(test_content, input_dim[i])
#print (len(new_content))
test_normal = new_content[10001:-1]
new_content = new_content[:10000]
print (len(new_content))
print(len(test_normal))

10000
1852


In [6]:
import random
random.seed(10)
iter = 0
def next_batch(batch_size):
    i = random.randint(0,len(new_content)-batch_size-1)
    return new_content[i:i+batch_size]
    #if((iter+batch_size)>=len(new_content)):
    #    return[0:batch_size]
    #    #iter =+ batch_size
    #    con = new_content[iter:iter+batch_size]
    #else:
    #    iter=batch_size
    #    return new_content[]
    #return con

In [7]:
con = next_batch(10)
#print (con)

In [8]:
hidden_encoder_dim = 200
hidden_decoder_dim = 200
latent_dim = 10
lam = 0.01

In [9]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.001)
  return tf.Variable(initial)

In [10]:
def bias_variable(shape):
  initial = tf.constant(0., shape=shape)
  return tf.Variable(initial)

In [11]:
x = tf.placeholder("float", shape=[None, input_dim[i]])
print ("shape: ", x.get_shape())
l2_loss = tf.constant(0.0)

shape:  (?, 20)


In [12]:
W_encoder_input_hidden = weight_variable([input_dim[i],hidden_encoder_dim])
b_encoder_input_hidden = bias_variable([hidden_encoder_dim])
l2_loss += tf.nn.l2_loss(W_encoder_input_hidden)

In [13]:
# Hidden layer encoder
hidden_encoder = tf.nn.relu(tf.matmul(x, W_encoder_input_hidden) + b_encoder_input_hidden)

W_encoder_hidden_mu = weight_variable([hidden_encoder_dim,latent_dim])
b_encoder_hidden_mu = bias_variable([latent_dim])
l2_loss += tf.nn.l2_loss(W_encoder_hidden_mu)

In [14]:
# Mu encoder
mu_encoder = tf.matmul(hidden_encoder, W_encoder_hidden_mu) + b_encoder_hidden_mu

W_encoder_hidden_logvar = weight_variable([hidden_encoder_dim,latent_dim])
b_encoder_hidden_logvar = bias_variable([latent_dim])
l2_loss += tf.nn.l2_loss(W_encoder_hidden_logvar)

In [15]:
# Sigma encoder
logvar_encoder = tf.matmul(hidden_encoder, W_encoder_hidden_logvar) + b_encoder_hidden_logvar

In [16]:
# Sample epsilon
epsilon = tf.random_normal(tf.shape(logvar_encoder), name='epsilon')

In [17]:
# Sample latent variable
std_encoder = tf.exp(0.5 * logvar_encoder)
z = mu_encoder + tf.multiply(std_encoder, epsilon)

W_decoder_z_hidden = weight_variable([latent_dim,hidden_decoder_dim])
b_decoder_z_hidden = bias_variable([hidden_decoder_dim])
l2_loss += tf.nn.l2_loss(W_decoder_z_hidden)

In [18]:
# Hidden layer decoder
hidden_decoder = tf.nn.relu(tf.matmul(z, W_decoder_z_hidden) + b_decoder_z_hidden)

W_decoder_hidden_reconstruction = weight_variable([hidden_decoder_dim, input_dim[i]])
b_decoder_hidden_reconstruction = bias_variable([input_dim[i]])
l2_loss += tf.nn.l2_loss(W_decoder_hidden_reconstruction)

In [19]:
KLD = -0.5 * tf.reduce_sum(1 + logvar_encoder - tf.pow(mu_encoder, 2) - tf.exp(logvar_encoder), reduction_indices=1)

x_hat = tf.matmul(hidden_decoder, W_decoder_hidden_reconstruction) + b_decoder_hidden_reconstruction
BCE = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(logits=x_hat, labels=x), reduction_indices=1)

loss = tf.reduce_mean(BCE + KLD)

regularized_loss = loss + lam * l2_loss

loss_summ = tf.summary.scalar("lowerbound", loss)
train_step = tf.train.AdamOptimizer(0.00001).minimize(regularized_loss)

In [20]:
# add op for merging summary
summary_op = tf.summary.merge_all()

# add Saver ops
saver = tf.train.Saver()

n_steps = int(1e4)
batch_size = 100

In [21]:
abnormal_list = []
with open('/media/nikhil/Data/Experiments/ECG Problem/sample rr interval data/line_count_rr_with_category.txt', 'r') as fr:
    rr_name = fr.readlines()[0].split('\r')[:-2]
    #print len(rr_name)
    for names in rr_name:
        name = names.split(' ')
        if(name[2]=="Abnormal" and int(name[0])<25 and int(name[0])>19):
            abnormal_list.append(name)
print(len(abnormal_list))

32


In [22]:
test_abnormal = []
for i, name in enumerate(abnormal_list):
    #print(name)
    with open('/media/nikhil/Data/Experiments/ECG Problem/sample rr interval data/'+name[1], 'r') as rr:
        l = rr.readlines()
        #for i, x in enumerate(l):
        l = [a.replace('\tN\n', '') for a in l]
        #print (l)
        test_abnormal.append(l)
    #abnormal_list[i] = ab_list[1:21]

In [23]:
#for test_ab in test_abnormal:
    #print (len(test_ab))

In [24]:
for i, test_ab in enumerate(test_abnormal):
    test_abnormal[i] = test_ab[:20]

In [25]:
#for test_ab in test_abnormal:
    #print (len(test_ab))

In [26]:
test_set = []
test_label = []
test_set = test_normal + test_abnormal
test_label = [1]* len(test_normal) + [0]*len(test_abnormal)

In [27]:
print (len(test_set), len(test_label))

1884 1884


In [28]:
with tf.Session() as sess:
    summary_writer = tf.summary.FileWriter('experiment',
                                           graph=sess.graph)
    if os.path.isfile("/media/nikhil/Data/Experiments/ECG Problem/VAE-TensorFlow-master/save/model.ckpt"):
        print("Restoring saved parameters")
        saver.restore(sess, "/media/nikhil/Data/Experiments/ECG Problem/VAE-TensorFlow-mastersave/model.ckpt")
    else:
        print("Initializing parameters")
        sess.run(tf.global_variables_initializer())
        
    for step in range(1, n_steps):
        #y = np.hstack(content[3])
        #print(y.shape)
        batch = next_batch(batch_size)
        #print (batch)
        #batch = np.array([np.array(con) for con in content[3] if len(con)==70])
        #for ba in batch:
        # print("Shape: ", ba.shape, "  ", ba)
        #print(batch.shape)
        #assert all(x.shape == batch for x in batch)
        feed_dict = {x: batch}
        #print (feed_dict)
        _, cur_loss, summary_str = sess.run([train_step, loss, summary_op], feed_dict=feed_dict)
        summary_writer.add_summary(summary_str, step)
        
        if step % 500 == 0:
            save_path = saver.save(sess, "/media/nikhil/Data/Experiments/ECG Problem/VAE-TensorFlow-master/save/model.ckpt")
            print("Step {0} | Loss: {1}".format(step, cur_loss))
    
    norm_res = []
    abnorm_res = []
    for dat in test_normal:
        norm_res.append(sess.run(loss, feed_dict={x:[dat]}))
    for dat in test_abnormal:
        abnorm_res.append(sess.run(loss, feed_dict={x:[dat]}))
        
    correct = 0
    incorrect = 0
    #for res in norm_res:
        #if(res>12.5):
        #    incorrect =+1
        #else:
        #    correct =+1 
    for res in abnorm_res:
        if(res>12.5):
            correct =correct+1
        else:
            incorrect =incorrect+1
            
    acc = correct/(correct+incorrect)
    print (correct)
    print (incorrect)
    print (acc)
    #feed_dict = {x:test_abnormal}
    #abnormal_error = sess.run(loss, feed_dict={x:test_abnormal})
    #print ("abnormal error: ", abnormal_error)
    #normal_error = sess.run(loss, feed_dict={x:test_normal})
    #print ("normal error: ", normal_error)

Initializing parameters
Step 500 | Loss: 13.7541122437
Step 1000 | Loss: 13.545083046
Step 1500 | Loss: 13.2571487427
Step 2000 | Loss: 12.9105873108
Step 2500 | Loss: 12.5100793839
Step 3000 | Loss: 12.3615627289
Step 3500 | Loss: 12.2945365906
Step 4000 | Loss: 12.5148267746
Step 4500 | Loss: 12.1308040619
Step 5000 | Loss: 12.693312645
Step 5500 | Loss: 12.0191879272
Step 6000 | Loss: 12.274471283
Step 6500 | Loss: 11.9967470169
Step 7000 | Loss: 12.461974144
Step 7500 | Loss: 11.9551830292
Step 8000 | Loss: 11.6970796585
Step 8500 | Loss: 11.7845067978
Step 9000 | Loss: 12.2278928757
Step 9500 | Loss: 12.1853275299
26
6
0.8125


In [29]:
for res in abnorm_res:
    print (res)

13.4262
13.1899
13.2622
14.0756
14.1074
13.8549
13.1181
13.9763
12.9841
13.1739
13.9356
13.6769
13.3901
14.0208
13.1151
11.3862
12.4156
12.9509
13.1185
15.323
14.1899
13.2342
12.2341
12.8121
12.6187
13.093
12.0012
11.4237
15.2129
12.1326
13.4211
16.2268


In [31]:
print(len(abnorm_res))
print(len(norm_res))

32
1852


In [48]:
correct = 0
incorrect = 0
for res in norm_res:
        if(res>=13):
            incorrect =incorrect+1
        else:
            correct =correct+1
acc = correct/(correct+incorrect)        
print (correct)
print (incorrect)
print (acc)

1525
327
0.82343412527


In [49]:
correct = 0
incorrect = 0
for res in abnorm_res:
        if(res<13):
            incorrect =incorrect+1
        else:
            correct =correct+1
acc = correct/(correct+incorrect)        
print (correct)
print (incorrect)
print (acc)

22
10
0.6875
